In [6]:
# Import all relevant libraries
import warnings
def fxn(): 
	warnings.warn("deprecated",DeprecationWarning)

with warnings.catch_warnings( ):
    warnings.simplefilter("ignore")
    fxn( )
    
import numpy as np
import h5py

# Keras imports
import keras
from keras.models import Sequential
from keras.layers import Permute, Flatten, Softmax, Dense, Conv1D, Conv2D, Conv2DTranspose, AveragePooling2D, Activation, Reshape
from keras.backend import permute_dimensions
from keras.optimizers import Adam

In [26]:
# Load data from specific trial
def get_trial(trial_num):    
    trial = h5py.File('../data/A0' + str(trial_num) + 'T_slice.mat', 'r')
    X = np.copy(trial['image'])
    y = np.copy(trial['type'])
    y = y[0,0:X.shape[0]:1]
    y = np.asarray(y, dtype=np.int32)
    y -= 769                            # shift class labels to [0-3]
    X = np.nan_to_num(X)[:, :22, :]     # remove EOG channels
    return X, y

''' 
for trial_num in xrange(1, 9):
    X, y = get_trial(trial_num)
'''

# Get the data from the first person
X, y = get_trial(1)
X_train = X
y_train = keras.utils.to_categorical(y, num_classes=4)
print(X_train.shape, y_train.shape)

# The data for each trial is of the shape (288, 22, 1000)
#   There are 288 samples per trial (12 of each class per "run", 4 classes, 6 "runs" 
#                                   at different time periods of the day)
#   There are 22 electrodes from the EEG (represents spatial aspect of the signals)
#   There are 1000 time units (4 seconds of data, sampled at 250Hz). The first 250 units
#                                   are when no movement occurs (but the cue is heard) and
#                                   the next 750 units are when the movement occurs
# The labels for each trial belong in one of 4 classes
#   0 - left
#   1 - right
#   2 - foot
#   3 - tongue

(288, 22, 1000) (288, 4)


In [59]:
# Create CNN model

# input is of the form: (sample, spatial, temporal)
model = Sequential()

# Temporal convolution
model.add(Reshape((22, 1000, 1), input_shape=(22, 1000)))
model.add(Conv2D(filters=40, kernel_size=(1, 25), activation='elu', strides=1))
print(model.output_shape)

# Spatial convolution
model.add(Conv2D(filters=40, kernel_size=(22, 40), activation='elu', data_format="channels_first"))
print(model.output_shape)

# Mean pool
model.add(AveragePooling2D(pool_size=(1,75), strides=(1,15)))
print(model.output_shape)

# Dense layers
model.add(Flatten())
model.add(Dense(units=400, kernel_initializer='glorot_normal'))
model.add(Dense(units=200, kernel_initializer='glorot_normal'))
model.add(Dense(units=4, kernel_initializer='glorot_normal'))
print(model.output_shape)

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.adam(lr=0.01),
              metrics=['accuracy'])

(None, 22, 976, 40)
(None, 40, 955, 1)
(None, 40, 59, 1)
(None, 4)


In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
288/288 [==============================] - 63s 219ms/step - loss: 5.4196 - acc: 0.2431
Epoch 2/5
288/288 [==============================] - 66s 229ms/step - loss: 7.3567 - acc: 0.2257
Epoch 3/5
288/288 [==============================] - 60s 209ms/step - loss: 12.0886 - acc: 0.2639
Epoch 4/5
288/288 [==============================] - 63s 217ms/step - loss: 12.0886 - acc: 0.2500
Epoch 5/5
